In [1]:
import pandas as pd
import numpy as np
from datetime import datetime
from calendar import isleap
import json
import jellyfish
from numpyencoder import NumpyEncoder
pd.set_option('display.max_colwidth', -1)

<ipython-input-1-887dceb19a33>:8: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  pd.set_option('display.max_colwidth', -1)


# Parse weekends of teachers

In [2]:
def prettify_teachers_name(name):
    if "." not in name:
        splitted_name = [pt.strip() for pt in name.split(" ")]
        splitted_name[1] = splitted_name[1][0].upper()+"."
        splitted_name[2] = splitted_name[2][0].upper()+"."
        splitted_name[1] += splitted_name[2]
        splitted_name = splitted_name[:2]
        return " ".join(splitted_name)
    return name

def prettify_name(name):
    return str(name).strip().replace("\xa0", "").replace("\n", " ").replace("  ", " ")

def get_teachers_id_by_name(t_dict, name):
    for i, data in t_dict.items():
        if data["name"] == name:
            return i
    return None

def find_most_similar(name_list, name):
    most_similar = ""
    s_points = 1000000
    for val in name_list:
        cur_points = jellyfish.levenshtein_distance(name, val)
        if cur_points < s_points:
            s_points = cur_points
            most_similar = val
    return most_similar

In [3]:
# constants
YEAR = datetime.now().year
LAST_ID = 0
translate_month = {
    "Январь": "January",
    "Февраль": "February",
    "Март": "March",
    "Апрель": "April",
    "Май": "May",
    "Июнь": "June",
    "Июль": "July",
    "Август": "August",
    "Сентябрь": "September",
    "Октябрь": "October",
    "Ноябрь": "November",
    "Декабрь": "December"
}

february_shift = (29 if isleap(YEAR) else 28)

month_to_shift = {
    "January": 0,
    "February": 31,
    "March": 31 + february_shift,
    "April": 62 + february_shift,
    "May": 92 + february_shift,
    "June": 123  + february_shift,
    "July": 153  + february_shift,
    "August": 184  + february_shift,
    "September": 215  + february_shift,
    "October": 245  + february_shift,
    "November": 276  + february_shift,
    "December": 306  + february_shift
}

translate_param = {
    "кол-во дней": "days",
    "декада": "decade"
}

# read weekend schedule
weekends = pd.read_excel("./Приложение №5.xls")
weekends.drop(["Unnamed: 27", "Unnamed: 2"], axis=1, inplace=True)

# make new columns
new_columns = []
cur_month = ""
for month, param in zip(weekends.iloc[5][2:], weekends.iloc[6][2:]):
    if month is not np.nan:
        cur_month = translate_month[month]
    new_columns.append(cur_month + " " + translate_param[param])
    
new_columns = ["id", "name"] + new_columns

# get important data from schedule
weekends = weekends.iloc[7:23]
weekends = weekends.reset_index(drop=True)

# rename columns
weekends.columns = new_columns

# fill NaN values by zeros
weekends = weekends.fillna(0)

# fill weekend schedule of teachers who is represented in DataFrame named "weekends"

# resulting dict and other helpful things
teachers = {}
teacher_name_list = []
LAST_ID = weekends["id"].values[-1]

for idx in weekends.index:
    teacher = weekends.iloc[idx]
    
    teachers[str(teacher["id"])] = {
        "name": prettify_teachers_name(teacher["name"]),
        "weekends": ([0]*366 if isleap(YEAR) else [0]*365)
    }
    
    teacher_name_list.append(teachers[str(teacher["id"])]["name"])
    
    t_vals = list(teacher.items())[2:]
    for i in range(1, len(t_vals), 2):
        col = t_vals[i][0]
        starting_decade = t_vals[i][1]
        days_of_weekend = t_vals[i-1][1]
        if col == "id" or col == "name":
            continue
            
        start = month_to_shift[col.split()[0]] + starting_decade*10
        end = start + days_of_weekend
        
        for day_i in range(start, end):
            teachers[str(teacher["id"])]["weekends"][day_i % (366 if isleap(YEAR) else 365)] = 1

# Parse programs

In [4]:
programs = {}
progs = pd.read_excel("./Приложение №2.xlsx", sheet_name="параметры программ")
progs = progs.rename({"№ п\п": "id", "Учебная программа": "name"}, axis=1)

In [5]:
progs["Особенности проведения программы"] = progs["Особенности проведения программы"].fillna("нет")
progs["Обучение в СДО, часов"] = progs["Обучение в СДО, часов"].fillna("-")
progs["Количество групп в план-графике в 2020"] = progs["Количество групп в план-графике в 2020"].fillna("0")
progs["Количество сотрудников ВВСС к обучению в 2020"] = progs["Количество сотрудников ВВСС к обучению в 2020"].fillna("0")

In [6]:
req = input()
if req == "y":
    f = open("./parsed_docx_ref.json", "r")
    prog_topics = json.load(f)
    f.close()

    tps = list(prog_topics.keys())

    for name in tps:
        n = find_most_similar(progs["name"].values, name)
        prog_topics[name]["name"] = n
        tmp = prog_topics[name]
        del prog_topics[name]
        prog_topics[int(progs.loc[progs["name"] == n, "id"].values[0])] = tmp

    f = open("./parsed_docx_ref.json", "w")
    json.dump(prog_topics, f)
    f.close()

n


In [7]:
cur_disc = ""
for val in progs.values:
    if val[1] is not np.nan:
        cur_disc = prettify_name(val[1])
    if val[9] == 0:
        continue
    prefs = val[3].split(",")
    _classes = prefs[2].split(";")
    if _classes[0] == "":
        _classes = []
        
    programs[int(val[0])] = {
        "name": prettify_name(val[2]),
        "discipline": cur_disc,
        "classes": _classes,
        "time": int(val[4]),
        "training": {
            "class": float((val[5] if str(val[5]).replace("\xa0", "") != "-" else 0)),
            "practice": float((val[6] if str(val[6]).replace("\xa0", "") != "-" else 0)),
            "distance": float((val[7] if str(val[7]).replace("\xa0", "") != "-" else 0)),
        },
        "people": int(val[10])
    }

# Parse preferences

In [8]:
f = open("./parsed_docx_ref.json", "r")
prog_topics = json.load(f)
f.close()

# read dataset with preferences
prefs = pd.read_excel("./Приложение №2.xlsx", sheet_name="параметры преподавателей")
prefs.drop([19], inplace=True)
prefs = prefs.rename({
    "Табельный номер преподавателя": "id",
    "Преподаватель": "name"
}, axis=1)


# create blank records of missing teachers
for name in prefs["name"].values:
    if name not in teacher_name_list:
        LAST_ID += 1
        teachers[str(LAST_ID)] = {
            "name": prettify_teachers_name(name),
            "weekends": ([0]*366 if isleap(YEAR) else [0]*365)
        }
    # making a column with ids
    prefs.loc[prefs["name"] == name, "id"] = get_teachers_id_by_name(teachers, name)

In [9]:
for ii in prefs["id"]:
    i = str(ii)
    teachers_discs = prettify_name(prefs.loc[prefs["id"] == i, "Дисциплина"].values[0]).split(";")
    teachers_progs = prettify_name(prefs.loc[prefs["id"] == i, "Учебные программы"].values[0]).split(";")
    
    teachers_discs = [prettify_name(d) for d in teachers_discs]
    teachers_progs = [int(p) for p in teachers_progs]
    
    # programs and disciplines
    teachers[i]["disciplines"] = []
    for disc in teachers_discs:
        disc_ps = []
        for prog_id in programs.keys():
            if disc == programs[prog_id]["discipline"]:
                disc_ps.append(prog_id)
        teachers[i]["disciplines"].append({
            "name": disc,
            "programs": {prog: [i for i in range(1, len(list(prog_topics[str(prog)].keys()))-2)] for prog in teachers_progs if prog in disc_ps and str(prog) in prog_topics.keys()}
        })

    # priority
    teachers_prior = prettify_name(prefs.loc[prefs["id"] == i, "Приоритет при распределении"].values[0]).split(",")
    
    teachers[i]["priority"] = {
        "theory": 1,
        "practice": 1,
        "lessons": [],
    }
    
    if teachers_prior[0] != "":
        teachers[i]["priority"]["theory"] = int(teachers_prior[0])
    
    if teachers_prior[1] != "":
        teachers[i]["priority"]["practice"] = int(teachers_prior[1])
    
    if teachers_prior[2] != "":
        prior_progs = [int(p) for p in teachers_prior[2].split(";")]
        for prog_id in teachers_progs:
            if prog_id in prior_progs:
                teachers[i]["priority"]["lessons"].append((prog_id, int(teachers_prior[3])))
            else:
                teachers[i]["priority"]["lessons"].append((prog_id, int(teachers_prior[3])+1))
    
    # topics
    
    f = open("./parsed_docx_ref.json", "r")
    prog_topics = json.load(f)
    f.close()
    
    teachers_topics = prefs.loc[prefs["id"] == i, "Может проводить занятия по темам"].values[0]
    if teachers_topics is not np.nan:
        teachers_topics = prettify_name(teachers_topics).split(";")
        for disc in teachers_discs:
            disc_ps = []
            for prog_id in programs.keys():
                if disc == programs[prog_id]["discipline"]:
                    disc_ps.append(prog_id)
        for con in teachers_topics:
            scon = con.split(":")
            if scon[0][:2] == "!>":
                for d in teachers[i]["disciplines"]:
                    if int(scon[1]) in d["programs"].keys():
                        for k in range(int(scon[0][2:])+1, len(list(prog_topics[scon[1]].keys()))-2):
                            d["programs"][int(scon[1])].remove(k)
            elif scon[0][:1] == ">":
                for d in teachers[i]["disciplines"]:
                    if int(scon[1]) in d["programs"].keys():
                        try:
                            d["programs"][int(scon[1])] = [k for k in range(int(scon[0][1:])+1, len(list(prog_topics[int(scon[1])].keys()))-2)]
                        except KeyError:
                            d["programs"][int(scon[1])] = [k for k in range(int(scon[0][1:])+1, len(list(prog_topics[scon[1]].keys()))-2)]
    else:
        for prog in teachers_progs:
            try:
                teachers[i]["programs"][prog] = [k for k in range(1, len(list(prog_topics[prog].keys()))-2)]
            except KeyError:
                pass

# Parse calendar

In [10]:
calendar = pd.read_csv("./calendar.csv")

calendar = calendar.iloc[21]

calendar = calendar[1:-5]

wkds_month = {}
for month, days in calendar.items():
    wkds_month[translate_month[month]] = []
    for day in days.split(","):
        if "*" not in day:
            wkds_month[translate_month[month]].append(int(day.replace("+", "")))

# нет графика сменности на 2020
for month, days in wkds_month.items():
    for teacher in teachers.keys():
        for day in days:
            teachers[teacher]["weekends"][month_to_shift[month]+int(day)-1] = 1

# SAVE

In [11]:
f1 = open("./teachers.json", "w")
json.dump(teachers, f1, cls=NumpyEncoder)

f2 = open("./parsed_programs.json", "w")
json.dump(programs, f2, cls=NumpyEncoder)

f1.close()
f2.close()